In [1]:
import pandas as pd

In [2]:
df_120text = pd.read_csv("https://raw.githubusercontent.com/corazzon/web_scraping/master/seoul-120/seoul-120-text.csv")
df_120text[:3]

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485


In [3]:
df_120text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645 entries, 0 to 2644
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      2645 non-null   int64 
 1   분류      2645 non-null   object
 2   제목      2645 non-null   object
 3   내용      2645 non-null   object
 4   내용번호    2645 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 103.4+ KB


In [4]:
stopwords = []

In [5]:
from konlpy.tag import Okt
okt = Okt()
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [6]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [11]:
df_120text_features = tfidfVectorizer.fit_transform(df_120text['내용'])
df_120text_array = df_120text_features.toarray()
df_120text_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
quest_result_list = cosine_similarity(df_120text_array[1772].reshape(1,-1),df_120text_array)
quest_result_list

array([[0.06642318, 0.00122671, 0.00490687, ..., 0.0108573 , 0.02590003,
        0.02998956]])

In [14]:
import numpy as np

In [15]:
quest_result_list = [item for sublist in quest_result_list for item in sublist]
df_similar = pd.DataFrame(quest_result_list, columns=['유사도'])

In [16]:
df_quest = pd.concat([df_120text,df_similar],axis=1)
df_quest

,번호,분류,제목,내용,내용번호,유사도
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,0.066423
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,0.001227
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485,0.004907
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492,0.029660
4,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798,0.015529
...,...,...,...,...,...,...
2640,5,주택도시계획,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...,2894353,0.030988
2641,4,환경,수돗물의 톤당 원가는?,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다,19353862,0.000000
2642,3,복지,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...,19699584,0.010857
2643,2,교통,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...,2898293,0.025900


In [21]:
df_quest_sort = df_quest.sort_values(by='유사도', ascending=False)
df_quest_sort [:6]

,번호,분류,제목,내용,내용번호,유사도
1772,873,경제,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까?,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까재결신청 이후 재결신청...,2897109,1.000000
1775,870,경제,도시계획시설 재결신청은 어떻게 합니까?,도시계획시설 재결신청은 어떻게 합니까재결신청은 소유자 및 사업시행자가 모두 할 수 ...,2897108,0.752630
1771,874,경제,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요?,도시계획시설부지의 재결과정에서는 의견을 제출할 수 없는지요재결신청 이후 신청서 및 ...,2897110,0.613373
1780,865,경제,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요?,도시계획시설에 편입된 보상물건의 재결신청 청구는 몇일 이내에 해야 하나요사업인정 고...,2897106,0.569272
1768,877,경제,도시계획시설부지에 포함된 건물보상금액의 불만입니다.,도시계획시설부지에 포함된 건물보상금액의 불만입니다사업시행자로 하여금 재결신청을 청구...,2897114,0.540980
1767,878,경제,도시계획시설 부지의 이의재결이란 무엇입니까?,도시계획시설 부지의 이의재결이란 무엇입니까수용재결에 대한 이의가 있는 경우 중앙토지...,2897115,0.494651
